# Lab 01 · FastAPI, Vite, and Git

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A FastAPI backend is scaffolded with health and echo routes.
- A Vite React frontend is outlined with modern tooling defaults.
- A Git repository is initialized with environment templates.

## What will be learned
- The structure of a basic FastAPI service is reviewed.
- Development-time CORS settings are configured.
- Vite-powered React scaffolding steps are rehearsed.
- Git initialization practices are reinforced.

## Prerequisites & install
The following tools are needed for this lab. Install them locally and keep all project-specific environments inside this repository so nothing leaks into the rest of your system.

| Tool | Why we use it in this lab | Download or docs |
| --- | --- | --- |
| Python 3.10+ | Runs the FastAPI backend and virtual environment tooling. | [python.org/downloads](https://www.python.org/downloads/) |
| Node.js & npm | Provides the JavaScript runtime and package manager required by Create React App. | [nodejs.org](https://nodejs.org/) |
| Git | Tracks changes and enables pushing your work to GitHub. | [git-scm.com/downloads](https://git-scm.com/downloads) |
| Create React App (npx) | Bootstraps the React frontend scaffold used in the lab. | [create-react-app.dev](https://create-react-app.dev/docs/getting-started/) |
| FastAPI | Python web framework for building the backend API. | [fastapi.tiangolo.com](https://fastapi.tiangolo.com/) |
| Uvicorn | ASGI server that runs the FastAPI app locally. | [www.uvicorn.org](https://www.uvicorn.org/) |
| Pydantic | Validates request/response models in the FastAPI backend. | [docs.pydantic.dev](https://docs.pydantic.dev/) |
| python-dotenv | Loads environment variables from a `.env` file for local secrets management. | [pypi.org/project/python-dotenv](https://pypi.org/project/python-dotenv/) |
| google-generativeai | Client library for Google Generative AI integrations explored later in the course. | [pypi.org/project/google-generativeai](https://pypi.org/project/google-generativeai/) |
| faiss-cpu | Provides vector similarity search utilities used by the AI features. | [pypi.org/project/faiss-cpu](https://pypi.org/project/faiss-cpu/) |
| numpy | Supports numerical operations required by FAISS and data processing. | [numpy.org](https://numpy.org/install/) |

Before installing dependencies, change into `ai-web/backend`, create (if needed) and activate the `.venv` virtual environment there, and scaffold the React app inside `ai-web/frontend` so every dependency stays scoped to this project folder.

```bash
python --version
node --version
npm --version
git --version

# Backend environment preparation (local execution only)
cd ai-web/backend
python -m venv .venv
# Windows: .venv\Scripts\activate
# Unix/macOS:
. .venv/bin/activate
pip install fastapi uvicorn[standard] pydantic python-dotenv google-generativeai faiss-cpu numpy

# Frontend creation (Create React App; no Vite)
cd ../../
npx create-react-app frontend
cd frontend
npm install
```

## Step-by-step tasks
Each step is executed locally so backend keys remain protected.

### Step 1: Backend folder layout
A backend folder is created and populated with required files.

In [ ]:
from pathlib import Path
base = Path("ai-web/backend")
(base / "app").mkdir(parents=True, exist_ok=True)
(base / "app" / "__init__.py").write_text("
")
(base / ".env.example").write_text('# Environment variables (never commit real keys)
GEMINI_API_KEY=
')
(base / "requirements.txt").write_text('''fastapi
uvicorn[standard]
pydantic
python-dotenv
google-genai
faiss-cpu
numpy
''')
(base / "app" / "main.py").write_text('''from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_methods=["*"],
    allow_headers=["*"],
)


class EchoIn(BaseModel):
    msg: str


@app.get("/health")
def health():
    return {"status": "ok"}


@app.post("/echo")
def echo(payload: EchoIn):
    return {"msg": payload.msg}
''')
print("Backend scaffold was written under ai-web/backend.")

### Step 2: Frontend placeholders
Frontend placeholders are positioned so Vite React components (`src/App.jsx`, `src/main.jsx`) can be customized while the generated `vite.config.js` continues to manage dev server defaults.

In [ ]:
from pathlib import Path
src = Path("ai-web/frontend/src")
(src / "lib").mkdir(parents=True, exist_ok=True)

(src / "lib" / "api.js").write_text('''const BASE = import.meta.env.VITE_API_BASE || "http://localhost:8000";

export async function post(path, body) {
  const res = await fetch(`${BASE}${path}`, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify(body)
  });
  if (!res.ok) {
    throw new Error(`HTTP ${res.status}`);
  }
  return res.json();
}
''')
(src / "App.jsx").write_text('''import { useState } from 'react';
import { post } from './lib/api';

function App() {
  const [msg, setMsg] = useState('hello');
  const [response, setResponse] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  async function handleSend() {
    setLoading(true);
    setError('');
    try {
      const json = await post('/echo', { msg });
      setResponse(json.msg);
    } catch (err) {
      setError(String(err));
    } finally {
      setLoading(false);
    }
  }

  return (
    <main style={{ padding: 24 }}>
      <h1>Lab 1 — Echo demo</h1>
      <input value={msg} onChange={(event) => setMsg(event.target.value)} />
      <button onClick={handleSend} disabled={loading}>Send</button>
      {loading && <p>Loading…</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}
      <pre>{response}</pre>
    </main>
  );
}

export default App;
''')
(src / "main.jsx").write_text('''import React from 'react';
import ReactDOM from 'react-dom/client';
import App from './App.jsx';

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>,
);
''')
print("Frontend placeholders were written under ai-web/frontend/src.")

### Step 3: Git initialization
Git is initialized locally so changes can be tracked.

```bash
cd ai-web
git init
git add .
git commit -m "Lab 1 scaffold"
```

## Validation / acceptance checks
```bash
# locally
curl http://localhost:8000/health
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"hello"}'
```
- HTTP 200 responses include status "ok" and the echoed payload.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A README entry is expanded to document backend and frontend start commands.
- A GitHub repository is connected for remote backups.